# HW 2: Wrangling

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

Work is done below answers.

1.1 The og Price was stored as an obj as prices >999 contained commas (ex: 1,000), which prevented pd from interpreting the cols as numeric. I cleaned the variable by converting it to a str, removing commas, and then converting it to a numeric variable usimg pd.to_numeric. After cleaning, I verified that there were zero missing values.

1.2 CSV had 1,631 'Yes', 1,446 'No', and 9,848 missing as it had hinted. This is a huge concern as 3/4 of the values are missing --> 76.2% are mising. This is concerning as most incidents do not record whether an injury occured. Cross-tabulating the cleaned injury variable with force_type shows that missingness is not random. For example, certain force types like Bodily Force and Chemical Irritant have disproportionately high levels of missing injury data (while more severe or specific force types are more likely to have injury status recorded.) This usggests that conclusions about injury rates by force type should be interpreted cautiously.

1.3 The variable was 1 for released, 0 for not released, and 9 for unknown. I kept the 1 and 0 values and replaced the 9 with np.nan (missing data), which is suitable as a dummy value for analysis.

1.4 The ImposedSentenceAllChargeInContactEvent contains many missing values because sentences are only imposed when a defendant is convicted. By lookinf at SentenceTypeAllChargesAtConvictionInContactEvent, it is clear that many cases involve dismissals or other outcomes where no sentence exists. Then, the missing values in the sentence length variable are not missing at random, but instead reflect structural missingness. I cleaned the variable by coercing it to numeric and preserving missing values as NaN rather than replacing them with zeros.

In [ ]:
# 1.1 Cleaning the Price variable
import pandas as pd
import numpy as np

airbnb = pd.read_csv("data/airbnb_hw.csv")
# airbnb.shape, airbnb.columns

# airbnb["Price"].value_counts().head(20)

# weird = airbnb.loc[~airbnb["Price"].astype(str).str.match(r"^\d+$", na=False), "Price"]
# weird.head(30), weird.shape
# # --> found $ over 999 has commas (ex: 1,000) ie not good

airbnb["Price_clean"] = (
    airbnb["Price"]
    .astype(str)                # make everything string
    .str.replace(",", "", regex=False)  # remove commas
)

airbnb["Price_clean"] = pd.to_numeric(airbnb["Price_clean"], errors="coerce")
airbnb["Price_clean"].dtype, airbnb["Price_clean"].isna().sum()

In [ ]:
# 1.2
mn = pd.read_csv("data/mn_police_use_of_force.csv")
mn.shape, mn.columns
mn["subject_injury"].value_counts(dropna=False)

mn["subject_injury_clean"] = mn["subject_injury"]
mn["subject_injury_clean"].isna().mean() #0.76193...

# cross tabulate with force_type
pd.crosstab(
    mn["force_type"],
    mn["subject_injury_clean"],
    dropna=False
)

In [ ]:
#1.3 
justice = pd.read_parquet("data/justice_data.parquet")
justice.shape, justice.columns

justice["WhetherDefendantWasReleasedPretrial"].value_counts(dropna=False)
justice["ReleasedPretrial_clean"] = justice["WhetherDefendantWasReleasedPretrial"]

justice["ReleasedPretrial_clean"] = justice["ReleasedPretrial_clean"].replace({
    1: 1,
    0: 0,
    9: np.nan
})
justice["ReleasedPretrial_clean"].value_counts(dropna=False)

In [ ]:
#1.4
justice["ImposedSentenceAllChargeInContactEvent"].value_counts(dropna=False).head(20)
justice["SentenceTypeAllChargesAtConvictionInContactEvent"].value_counts(dropna=False)


justice["ImposedSentence_clean"] = pd.to_numeric(
    justice["ImposedSentenceAllChargeInContactEvent"],
    errors="coerce"
)

justice["ImposedSentence_clean"].isna().sum()


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

2.3 The year variable was stored as a numeric var with decimal values (such as 2015.0) and imcludeed invalid values (such as 0). Coerced to numeric, trated 0 as missing, converted all to ints. From there, the valid years ranged broadly (as low as 5) up to 2026. Restricting the year >= 1940 gives a cleaned range with a trend of attacks increasing over time (with noise year to year). The long-run pattern rises considerably in the late 20th century. The last few years may look lower from incomplete reporting.

2.4 Cleaned the Age to numeric so non-numeric ages become missing and the histogram shows that victims are concentrated in younger ages (teens through 30s) with a right-skewed distribution and a long tail to older ages (max observed is 87).

2.5 After cleaning the M, F, and unknown, the proportion male is 0.8028 (around 80.3% male)

2.6 Standardized Type and recorded everything other than clear Provoked and  Unprovoked (such as Invalid, Watercraft, etc.) to Unknown. THe proportion unprovoked is 0.7386 (around 73.9% unprovoked). Unprovoked: 5222, Provoked: 644, and Unknown: 1204.

2.7 Y: 4939, Y: 1488, Unknown: 643

2.8 Sharks are more likely to launch unprovoked atacks with male victims (83.6%) than female victimes (13.2%).  

The attack is more likely to be fatal when unprovoked than provoked. Only about 3% of provoked attacks result in fatalitlies compared to roughtly 24% of unprovoked attacks.

Attacks involving male victims are more likely to be fatal than those involving female victimes -- approximately 22% of attacks on males are fatal compared to about 15% for females.

I'm terrified of being in the ocean. I went to Hawaii over winter break. I thought I'd be brave enough to parasail, but after being shown videos of sharks in clear waters over people parasailing, I decided my life is much too precious. However, seeing the data suggest that fatal attacks are relatively rare and that risk is strongly associated with human behavior and exposure. Overall, sharks appear to pose a low-probability risk rather than an indiscrimate threat. Still, I like to keep myself at peace on dry land, where there are no sharks. :D

2.9 Approximately 10.7% of recorded shark attacks appear to involve whtie sharks, based on species descriptions contained the word 'white'.


In [ ]:
sharks = pd.read_excel("data/GSAF5.xls")
sharks.shape
sharks.head()
sharks.isna().all() # found no columns are completely empty

empty_cols = sharks.columns[sharks.isna().all()]
empty_cols
sharks = sharks.drop(columns=empty_cols)
sharks.shape

sharks["Year"].value_counts(dropna=False).head(20)
sharks["Year"].describe() # found floats

sharks["Year_clean"] = pd.to_numeric(sharks["Year"], errors="coerce")
sharks.loc[sharks["Year_clean"] == 0, "Year_clean"] = np.nan
sharks["Year_clean"] = sharks["Year_clean"].astype("Int64")

sharks["Year_clean"].describe()
sharks_1940 = sharks[sharks["Year_clean"] >= 1940]
sharks_1940["Year_clean"].min(), sharks_1940["Year_clean"].max()

attacks_per_year = (
    sharks_1940
    .groupby("Year_clean")
    .size()
)

attacks_per_year.head()
ttacks_per_year.plot(title="Shark Attacks per Year (since 1940)")

sharks["Age"].value_counts(dropna=False).head(20)
sharks["Age_clean"] = pd.to_numeric(sharks["Age"], errors="coerce")

sharks["Age_clean"].describe()


sharks["Age_clean"].plot(
    kind="hist",
    bins=30,
    title="Distribution of Shark Attack Victim Ages"
)

sharks["Sex"].value_counts(dropna=False)
sharks["Sex_clean"] = sharks["Sex"].astype(str).str.strip().str.upper()

sharks["Sex_clean"] = sharks["Sex_clean"].replace({
    "M": "M",
    "F": "F"
})

# Everything else--> Unknown
sharks.loc[~sharks["Sex_clean"].isin(["M", "F"]), "Sex_clean"] = "Unknown"

sharks["Sex_clean"].value_counts()

(sharks["Sex_clean"] == "M").mean()

sharks["Type"].value_counts(dropna=False)
sharks["Type_clean"] = (
    sharks["Type"]
    .astype(str)
    .str.strip()
    .str.capitalize()
)

sharks["Type_clean"] = sharks["Type_clean"].replace({
    "Provoked": "Provoked",
    "Unprovoked": "Unprovoked"
})

sharks.loc[
    ~sharks["Type_clean"].isin(["Provoked", "Unprovoked"]),
    "Type_clean"
] = "Unknown"

sharks["Type_clean"].value_counts()
(sharks["Type_clean"] == "Unprovoked").mean()

sharks["Fatal Y/N"].value_counts(dropna=False)
sharks["Fatal_clean"] = (
    sharks["Fatal Y/N"]
    .astype(str)
    .str.strip()
    .str.upper()
)

sharks["Fatal_clean"] = sharks["Fatal_clean"].replace({
    "Y": "Y",
    "N": "N"
})

sharks.loc[
    ~sharks["Fatal_clean"].isin(["Y", "N"]),
    "Fatal_clean"
] = "Unknown"

sharks["Fatal_clean"].value_counts()

# unprov att more likely in m or f
pd.crosstab(sharks["Type_clean"], sharks["Sex_clean"], normalize="index")

# att more likely fatal when prov or unprov
pd.crosstab(sharks["Type_clean"], sharks["Fatal_clean"], normalize="index")

# att more likely fatal for m or f
pd.crosstab(sharks["Sex_clean"], sharks["Fatal_clean"], normalize="index")


sharks.columns = sharks.columns.str.strip()
"Species" in sharks.columns, sharks.columns
(sharks["Species"].astype(str).str.lower().str.contains("white")).mean()


**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

3.1 Based on the abstract, this paper is about improving the proces of data cleaning by focusing on data tidying as a key component. It introduces the concept of tidy data, which follows a specefic structure wheree each variable is a column, each obs is a row, and each type of oberservational unit formas a table. It argues that the structgure makes messy datasets easier to clean, analyze, and model. It also allows analysis to rely on a small, consistent set of tools. THe benefits of tidy data are demosntrated through a case study that minimizes routine data manipulation. 

3.2 The "tidy data standard" by Wickham is intended to provide a consistent, reuseable way to organize data values within a dataset so analysts don't have to 'reinvent the wheel' each time they clean data. The design:
1. makes initial data cleaning and exploratory analysis easier
2. simplifies thedev and use of analysis tools that work well together
Tidy data also reduces the need to constantly translate or 'munge' outputs from one tool into inputs for another, which helps analysts focus on the the main problem without the logistics of replacing data.

3.2 The first sentence means that tidy datasets all share the same standardized structure, while messy datasets can be messy in many different ways. Because tidy data follow consistent rules, they are easier to recognize and work with, whereas messy datasets vary widely in their structure and problems. The second sentence means that tidy data organize rows and columns in a way that clearly reflects the meaning of the data. The physical structure of the dataset directly represents its semantics, making it easier to understand what each value, variable, and observation represents.


3.4 Wickham describes a value as a single measurement, usually a number or string. Each value belogns to both a variable and an observation. A variable contais all values that measure the same underlying attribute across units, such as height and temperature. An observation are values measured on the same unit across attributes, such as all measurements for one person or one experimental case.

3.5 Tidy data is aa standard way of mapping the meaning of a dataset to its structure. A dataset is tidy when each variable forms a column, each observation forms a row, and each type of obervational unit forms a table. Any other arrangement of the data is oncsidered messy. This structure makes it easier to extract variables and reduce errors. It's great for analyssi (especially in vectorized languages) as values from the same obervation are consistently aligned.


3.6 Five common problems:
1. col headers are values rather than var names
2. multiple vars are stored in one column
3. vars are stroed in both rows and columns
4. multiple types of observational unites are stored in the same table
5. a single observational unit is stored in multiple tables
The data in Table 4 is messy because the column headers represent values of the income var rather than var names, while religion is stored as a row var. Tis mixes up the vars and values across rows and column, so extracting variables in a consistent way becomes difficult. "Melting" a dataset is restructuring the data by turning columns that contain values into rows. Columns that are already variables are kept fixed, while the remaining columns are converted into two new variables, where one contains the former column names and the other contains their values.


3.7 Table 11 stores values across multiple columns representing the days of the month (d1, d2..) which means that var names and values are mixed together in the column structure, creating harder differentiation with variables and observations. Table 12(a) is molten because the dataset has been reshaped so that the day-specific columns have been converted into rows, producing variables for date, elemetn, and value. However, it is not fully tidy as teh element column still contains variable names (like 'tmax') rather than values.

3.8 Wickham describes a “chicken-and-egg” problem in which tidy data are only useful when paired with tools that support them, but tidy tools depend on tidy data structures. This interdependence can trap analysts in a local maximum where improving tools or data alone does not improve workflows. Wickham hopes that future work will build on the tidy data framework by exploring alternative data structures, developing better tools, and incorporating insights from human-centered design to make data cleaning and analysis easier.